In [15]:
import os
import re

import matplotlib.pyplot as plt
import pandas as pd

In [16]:
RAW_DATA_PATH = "../data/raw/"
PROCESSED_DATA_PATH = "../data/processed/"

## Очистка данных

### Данные пациентов (1 табл)

In [17]:
patients_df = pd.read_csv(os.path.join(RAW_DATA_PATH, "данные_пациентов.csv"))
patients_df.shape

(413799, 5)

In [18]:
patients_df = patients_df.drop_duplicates(keep="first")
patients_df.shape

(379246, 5)

Пациенты могли переезжать. Предполагаем, что id пациента отражают время регистрации => данные отранжированы по времени. И оставляем только последние записи для каждого пациента в каждом районе проживания.

In [19]:
patients_df = patients_df.sort_values("id_пациента").reset_index(drop=True)
patients_df = patients_df.drop_duplicates(subset=["id_пациента"], keep="last")
patients_df.shape

(379076, 5)

In [20]:
patients_df["дата_рождения"] = pd.to_datetime(
    patients_df["дата_рождения"],
    format="%d.%m.%Y",
    dayfirst=True,
    errors="coerce",
)

In [21]:
patients_df["пол"] = patients_df["пол"].str.lower()
patients_df["район_проживания"] = patients_df["район_проживания"].str.lower()
patients_df["регион"] = patients_df["регион"].str.lower()

In [22]:
patients_df.rename(columns={"id_пациента": "id"}, inplace=True)

In [23]:
patients_df.to_csv("../data/processed/patients.csv", index=False)

### Данные диагнозов

In [24]:
diagnoses_df = pd.read_csv("../data/raw/данные_диагнозы.csv")

In [25]:
diagnoses_df["код_мкб"] = diagnoses_df["код_мкб"].str.lower()
diagnoses_df["название_диагноза"] = diagnoses_df["название_диагноза"].str.lower()
diagnoses_df["класс_заболевания"] = diagnoses_df["класс_заболевания"].str.lower()
diagnoses_df = diagnoses_df.drop_duplicates(keep="first")

In [26]:
diagnoses_df.head()

,код_мкб,название_диагноза,класс_заболевания
0,a00,холера,некоторые инфекционные и паразитарные болезни
1,a00.0,"холера, вызванная холерным вибрионом 01, биова...",некоторые инфекционные и паразитарные болезни
2,a00.1,"холера, вызванная холерным вибрионом 01, биова...",некоторые инфекционные и паразитарные болезни
3,a00.9,холера неуточненная,некоторые инфекционные и паразитарные болезни
4,a01,тиф и паратиф,некоторые инфекционные и паразитарные болезни


In [27]:
diagnoses_df.to_csv("../data/processed/diagnoses.csv", index=False)

### Данные рецептов

In [28]:
recipes_df = pd.read_csv("../data/raw/данные_рецептов.csv")
recipes_df.head()

,id_пациента,дата_рецепта,код_диагноза,код_препарата,id_пациента.1
0,24419779,2022-03-31 11:52:30,F06,200900018,1226228
1,17890856,2019-06-04 18:41:10,I11.9,201300167,62857
2,18749669,2019-10-29 12:35:31,K86.1,201300027,1186134
3,31412579,2024-07-30 17:06:59,I67.8,202200024,1927759
4,21191943,2020-12-01 19:59:02,E10.9,201200267,1265025


In [29]:
recipes_df.isna().sum()

id_пациента         0
дата_рецепта     1178
код_диагноза        0
код_препарата       0
id_пациента.1       0
dtype: int64

In [30]:
recipes_df.dropna(inplace=True)

`id_пациента` не нужен

In [31]:
recipes_df = recipes_df.drop(columns=["id_пациента"])
recipes_df = recipes_df.rename(columns={"id_пациента.1": "id_пациента"})

ранжируем по дате выписки рецепта

In [32]:
recipes_df["дата_рецепта"] = pd.to_datetime(recipes_df["дата_рецепта"])
recipes_df = recipes_df.sort_values("дата_рецепта")

In [33]:
recipes_df["код_диагноза"] = recipes_df["код_диагноза"].str.lower()

удаляем дубликаты

In [34]:
recipes_df = recipes_df.drop_duplicates(keep="first")

In [36]:
recipes_df = recipes_df[recipes_df["код_диагноза"].isin(diagnoses_df["код_мкб"])]

In [39]:
recipes_df.head()

,дата_рецепта,код_диагноза,код_препарата,id_пациента
821956,2019-04-29 18:29:06,e10,201000730,80022
874569,2019-04-29 19:01:52,e10.6,201300134,1605354
423419,2019-04-29 19:02:50,e10.6,201000730,1605354
929897,2019-04-30 12:26:48,e10,201100021,1623781
522148,2019-04-30 12:49:14,e10.9,200900214,1301449


In [40]:
recipes_df.to_csv("../data/processed/recipes.csv", index=False)

### Данные препаратов

In [41]:
medication_df = pd.read_csv("../data/raw/данные_препараты.csv")

In [42]:
medication_df.isna().sum()

код_препарата           0
дозировка             116
Торговое название       0
стоимость            2201
Полное_название         0
dtype: int64

In [43]:
medication_df.columns = [
    "код_препарата",
    "дозировка",
    "торговое_название",
    "стоимость",
    "мета_информация",
]

In [44]:
medication_df["дозировка"] = medication_df["дозировка"].fillna("unknown").str.lower()
medication_df["стоимость"] = medication_df["стоимость"].fillna(-1).astype(int)
medication_df["мета_информация"] = medication_df["мета_информация"].str.lower()
medication_df["торговое_название"] = medication_df["торговое_название"].str.lower()

In [45]:
medication_df.head()

,код_препарата,дозировка,торговое_название,стоимость,мета_информация
0,200052200,1.5,галоперидол,12,"галоперидол, 1,5 мг № 50"
1,200052400,5,галоперидол,15,"галоперидол, 5 мг № 50, таблетки"
2,200059000,3.5,манинил,93,"манинил, 3,5 мг № 120, таблетки"
3,200165100,50,пури-нетол,525,"пури-нетол, 50 мг № 25, таблетки"
4,200194100,0.01,нифедипин,12,"нифедипин, 10 мг № 50, таблетки"


In [46]:
medication_df.to_csv("../data/processed/medication.csv", index=False)